In [ ]:
# Jupyter Notebook options

# Set the default plot width for Jupyter Notebook display
options(repr.plot.width = 16, repr.plot.height = 8)

# Datenbanken

Wir haben bisher verschiedene Datenquellen in R kennengelernt – wir haben Daten über Pakete geladen, z. B. mit `library(palmerpenguins)`, aber auch Datensätze aus dem lokalen Laufwerk `read_csv(csv_file)` geladen. Ein großer Teil aller Daten liegt allerdings in Datenbanken. Daher ist es wichtig zu wissen, wie wir auf Daten aus Datenbanken zugreifen können. Natürlich könnte man auch jedes Mal die Admin-Person einer Datenbank fragen, ob sie uns einen *snapshot* der Datenbank als `.csv` exportiert. Das wäre dann aber sehr aufwendig!

Daher werden wir in dieser Sitzung einen grundlegenden Einblick in SQL bekommen. SQL steht für *Structured Query Language* und ist eine der gängigsten Programmiersprachen für die Verwaltung und Operation von Datenbanken. SQL ist die *lingua franca* der Datenbanken und daher eine wichtige Sprache für Data Science Projekte. SQL kann jedoch auch sehr komplex werden – daher werde wir in dieser Sitzung nur die absoluten Basics von SQL kennenlernen.

In R gibt es analog zu `dplyr` das Paket `dbplyr`, mit dem wir die bereits bekannten Funktionen von `dplyr` in SQL übersetzen können. Zusätzlich werden wir wieder das `tidyverse` und das Paket `DBI` benutzen. `DBI` ist ein Interface, mit dem wir auf Datenbanken zugreifen und mit SQL Abfragen durchführen können. Außerdem benutzen wir das Paket `duckdb`, mit dem wir eine eigene Datenbank auf unserem Computer simulieren können. `nycflights13` benutzen wir wieder als Datensatz. Zur besseren Übersichtlichkeit und damit für euch transparent ist welche Funktion aus welchem Paket kommt, verwenden wir in diesem Skript oft die Schreibweise `paketname::funktionsname()`. Das ist normalerweise nicht notwendig.

Als erstes laden wir wieder die notwendigen Pakete:

In [ ]:
pacman::p_load(tidyverse,   # zum allgemeinen Datenhandling
               DBI,         # für SQL-Abfragen
               dbplyr,      # um dplyr-Funktionen in SQL zu "übersetzen"
               duckdb,      # zur Simulation einer Datenbank
               nycflights13 # als Beispieldaten
              )

## Datenbanken

Auf der einfachsten Ebene kann man sich eine Datenbank als eine Sammlung von Tabellen vorstellen. Jede Tabelle besteht dabei aus Spalten, Zeilen und Werten (analog zu einem Dataframe). Es gibt drei große Unterschiede zwischen Datenbanken und Dataframes:

* Datenbanktabellen werden auf einem Server gespeichert und können beliebig groß sein.
> Dataframes werden im Arbeitsspeicher gespeichert und sind dadurch in ihrer Größe begrenzt.
* Datenbanktabellen haben fast immer Indizes. Ähnlich wie das Inhaltsverzeichnis eines Buches ermöglicht ein Datenbankindex das schnelle Finden von interessanten Zeilen, ohne jede einzelne Zeile durchsehen zu müssen.
> Dataframes und Tibbles haben nicht zwingend Indizes!
* Die meisten klassischen Datenbanken sind darauf optimiert, Daten möglichst schnell abzurufen. Diese Datenbanken werden als zeilenorientiert bezeichnet, weil die Daten zeilenweise gespeichert werden. Dataframes in R sind spaltenweise aufgebaut (jede Spalte oder Variable ist in R ein Vektor und der Dataframe ist vom Prinzip her eine Liste in der verschidene Vektoren gleicher Länge miteinander verknüpft sind).

Datenbanken werden von Datenbankverwaltungssystemen (kurz DBMS) betrieben. In großen Organisationen oder Unternehmen gibt es viele verschiedene Datenbanken, die teilweise auch hierarchisch sortiert sein müssen. Dafür gibt es in SQL sogenannte hierarchische Strukturen, die die Organisation von Daten erleichtern soll. Diese Hierarchie ermöglicht eine effiziente Organisation und Verwaltung von Daten in komplexen Datenbanksystemen.

* eine **Tabelle** ist das grundlegende Element einer Datenbank und besteht aus Zeilen und Spalten. Jede Tabelle enthält Daten für bestimmte Themen. Z.B. könnte eine Tabelle namens "Prüfungen" Informationen wie Prüfungsdatum, Matrikelnummer und Note speichern. Eine andere Tabelle namens "Studierende" enthält dann vielleicht Matrikelnummer, Namen und Kontaktinformationen.
* ein **Schema** ist ein logischer Container für Tabellen, Ansichten (sogenannte Views) und andere Datenbankobjekte. Es wird verwendet, um Objekte innerhalb einer Datenbank zu organisieren und zu gruppieren. Mehrere Tabellen können unter demselben Schema gruppiert werden, und Schemata können dazu verwendet werden, den Zugriff auf die darin enthaltenen Datenbankobjekte zu steuern. Z. B. könnte ein Schema namens "Studierende" Tabellen wie "Persönliche Daten", "Prüfungen" und "Veranstaltungen" enthalten.
* ein **Katalog** ist eine noch höhere Ebene der Organisation und kann mehrere Schemata enthalten. Es ist auch möglich, dass ein Datenbanksystem mehrere Kataloge umfasst. Ein Katalog ist quasi wie eine große Bibliothek, und die Schemata sind die verschiedenen Abteilungen innerhalb dieser Bibliothek. Jeder Katalog kann seine eigenen Sicherheits-, Zugriffs- und Benutzereinstellungen haben.



## Mit einer Datenbank verbinden

Um sich in R mit einer Datenbank zu verbinden, werden zwei Pakete benötigt:

* Es wird immer das Paket `DBI` (Database Interface) benötigt. `DBI` enthält alle allgemeinen Befehle, um sich mit einer Datenbank zu verbinden, Daten zu schreiben oder zu laden, SQL-Abfragen durchzuführen usw.
* Zusätzlich wird je nach Datenbanktyp ein spezielles Paket benötigt. Dieses Paket übersetzt die speziellen Befehle je nach Datenbanktyp.

Im Folgenden sehen wir zwei Beispiele für RMariaDB und RPostgres (zwei verschiedene Typen von SQL-Datenbanken). Keine Sorge, das sind nur Beispiele! Sie dienen dazu, dass ihr grob schon mal gesehen habt, wie solcher Code aussehen könnte:

```
con <- DBI::dbConnect(
  RMariaDB::MariaDB(), 
  username = "foo"
)
con <- DBI::dbConnect(
  RPostgres::Postgres(), 
  hostname = "databases.mycompany.com", 
  port = 1234
)
```

### `duckdb`

In dieser Sitzung auf eine "richtige" Datenbank auf einem Server zuzugreifen, ist leider nicht möglich. Zum Glück gibt es für unsere Zwecke das Paket `duckdb`, mit dem wir eine Datenbank lokal simulieren können. Diese Datenbank lebt vollständig im Paket `duckdb` und wird bei jedem Schließen von R ebenfalls wieder geschlossen. Dadurch können wir erste Erfahrungen im Umgang mit Datenbanken sammeln!

Mit dem folgenden Befehl können wir uns mit `duckdb()` zunächst eine leere Datenbank anlegen und uns anschließend mit `dbConnect()` mit ihr verbinden.

In [ ]:
con <- duckdb::duckdb() %>%
    DBI::dbConnect()

Wenn wir mit einer Datenbank interagieren wollen, müssen wir zunächst eine Verbindung zu ihr herstellen. Die ganze "Kommunikation" mit der Datenbank läuft dann über diese Verbindung. Die Verbindung zur Datenbank haben wir hier `con` genannt. Wir sind ja etwas schreibfaul und das ist eine übliche Abkürzung für "connection".

> **`duckdb` permanent speichern**

>Es ist übrigens möglich, eine mit `duckdb` erstellte Datenbank auch **permanent** auf der eigenen Festplatte zu speichern. `duckdb` ist als high-performance database in der Lage, Gigabytes an Data mit großer Geschwindigkeit zu handeln. Wenn ihr in einem richtigen Data Science Projekt eine Datenbank braucht, könnt ihr daher gut auf `duckdb` zurückgreifen. Mithilfe des Arguments `dbdir = path_to_your_database` kann `duckdb` auf die Festplatte geschrieben werden. Das wollen wir in dieser Sitzung allerdings nicht machen, daher wird der Code hier nur kurz gezeigt.

>```con <- DBI::dbConnect(duckdb::duckdb(), dbdir = "path_to_your_database/duckdb")```

>Wenn ihr `duckdb` in einem echten Projekt benutzt, ist es außerdem sinnvoll, sich die Funktion `duckdb_read_csv()` anzuschauen. Mit dieser können csv-Dateien unkompliziert in die `duckdb`-Datenbank eingelesen werden.

## Daten in die Datenbank schreiben

Mit der Funktion `duckdb()`, die wir eben verwendet haben, wird jedes Mal eine neue Datenbank initialisiert. Es liegen also noch keine Daten darin. Um mit der Datenbank zu arbeiten, schreiben wir zwei Datensätze als Tabellen aus dem Paket `nycflights13` in die Datenbank. Das geht über den Befehl `dbWriteTable()`:

In [ ]:
con %>%
    dbWriteTable("flights", nycflights13::flights)

con %>%
    dbWriteTable("planes", nycflights13::planes)

### `DBI` basics

Wir können jetzt grundlegende Funktion von `DBI` benutzen, um Informationen über unsere Datenbank zu erhalten. Z. B. können wir mit `dbListTables()` alle Tabellen der Datenbank ausgeben oder mit `dbReadTable()` eine einzelne Tabelle auslesen. Die Funktion `as_tibble()` verwandelt dabei den zurückgegebenen `data.frame` von `dbReadTable()` in einen `tibble`, sodass wir wie gewohnt damit arbeiten können.

In [ ]:
con %>%
    dbListTables()

In [ ]:
con %>%
    dbReadTable("flights") %>%
    as_tibble() %>%
    print()

Wie wir sehen können, haben wir den bereits bekannten Datensatz `nycflights13::flights` erfolgreich in unsere Datenbank geschrieben. Wir können auch bereits eine kleine SQL-Abfrage mithilfe von `dbGetQuery()` benutzen, um Daten aus unserer SQL-Datenbank abzurufen. SQL ist
eine eigene Sprache, mit einer eigenen Syntax. Der Funktion `dbGetQuery()` kann man einfach einen Text übergeben, der in SQL geschreiben wurde:

In [ ]:
sql <- "
  SELECT flight, origin, dest, time_hour, dep_delay
  FROM flights 
  WHERE dep_delay > 100
"
con %>%
    DBI::dbGetQuery(sql) %>%
    as_tibble() %>%
    print()

Keine Sorge, wenn du bisher noch kein SQL gelesen hast! Wir werden im Folgenden einige grundsätzliche Funktionen kennenlernen. Wenn du dir den SQL-Code sorgfältig durchliest, wirst du vielleicht raten, dass der Code fünf Spalten aus dem `nycflights13::flights`Datensatz auswählt und alle Reihen zurückgibt, die mehr als 100 Minuten Verspätung haben.

## Das Zusammenspiel von SQL und `dbplyr`

Im nächsten Schritt werden wir `dbplyr` und damit auch weitere Aspekte von SQL kennenlernen. `dbplyr` ist ein sogenanntes `dplyr` **backend**. Das bedeutet, dass wir `dplyr`-Code schreiben können, der dann in SQL übersetzt wird. Das ist für uns sehr praktisch, da wir bereits bekannte Funktionen schreiben können und uns anschließend den übersetzten SQL-Code anschauen können.

Um `dbplyr` zu benutzen, müssen wir als erstes mithilfe von `tbl()` ein Objekt erzeugen, das einer Datenbanktabelle entspricht.

In [ ]:
flights_db <- tbl(con, "flights")
planes_db <- tbl(con, "planes")

> `flights_db` und `nycflights13::flights` enthalten jetzt zwar die gleichen Daten, sind aber zwei unterschiedliche Objekte! Das können wir uns gut mit der Funktion `class()` anschauen, die uns den Typ eines Objekts zurückgibt:

In [ ]:
class(flights_db)
class(flights)

Wie wir sehen können, ist `flights_db` eine `tbl_duckdb_connection`, während `flights` ein bereits bekannter `data.frame` ist.

Wir können jetzt wie gewohnt die `dplyr`-Befehle auf diesen Datenbank-Objekten ausführen. 

In [ ]:
flights_db %>%
    filter(dep_time > 100) 

Wenn wir an die Pipe die Funktion `show_query()` dranhängen, wird uns stattdessen der übersetzte SQL-Code angezeigt. Das ist praktiscch, weil wir so etwas über SQL lernen können:

In [ ]:
flights_db %>%
    filter(dep_time > 100) %>%
    show_query()

Da SQL-Datenbanken typischerweise sehr groß sind, sucht SQL nicht bei jeder Abfrage immer alle Zeilen heraus. Dafür müssen wir das explizit mithilfe des Befehls `collect()` angeben! Damit werden alle Ergebnisse als `tibble` zurückgegeben!

In [ ]:
flights_db %>%
    filter(dep_time > 100) %>%
    collect() %>%
    head(5) # print only the first 5 rows

Normalerweise werdet ihr `dbplyr` benutzen, um Daten aus SQL-Datenbanken auszulesen. Diese Datenbanken sind teilweise riesig, daher ist es wesentlich performanter (schneller), zunächst grundlegende Filter- und Aggregationsfunktionen durchzuführen. Dafür könnt ihr dann die bekannten Funktionen von `dplyr` benutzen. Sobald die Daten gefiltert und aggregiert sind und die Größe des Datensatzes geschrumpft ist, könnt ihr diese dann mithilfe von `collect()` in einen in-memory `tibble` laden und die Arbeit mit R fortsetzen.

### SQL basics

Die *top-level components* von SQL sind sogenannte **statements**. Häufig vorkommende statements sind `CREATE` für das Erstellen neuer Tabellen, `INSERT` für das Hinzufügen von Daten und `SELECT` für das Abfragen von Daten. Wir werden uns in dieser Übung auf `SELECT` beschränken, da wir in einem Data Science Projekt vor allem Daten abfragen werden.

Eine **Query** (Abfrage) besteht aus sogenannte **clauses**. Es gibt fünf wichtige clauses, die wir im folgenden kennenlernen werden: 
- `SELECT` = Welche Daten (Variablen) sollen ausgewählt werden?
- `FROM` = Woher sollen diese Daten kommen? In welcher Tabelle der Datenbank sind sie zu finden?
- `WHERE` = Sollen die Daten nach einem bestimmten Kriterium ausgewählt werden? (entspricht *filter*)
- `ORDER BY` = Sollen die Daten sortiert werden? (entspricht *arrange*)
- `GROUP BY` = Sollen die Daten irgendwie gruppiert werden? (entspricht *group_by*)

In jeder Query muss mindestens `SELECT` und `FROM` vorkommen. Die einfachste Abfrage ist es, jedes Element aus einer Tabelle mithilfe von `SELECT * FROM TABLE` abzufragen. Das `*` symbolisiert hierbei, dass wir alle Elemente abfragen wollen. Mit `show_query()` können wir uns diesen einfachsten SQL-Befehl, der eine ganze Tabelle zurück gibt, ausgeben lassen. Im Folgenden werden wir öfters `dbplyr`-Code ausführen und uns dann den in SQL übersetzten Befehl anzeigen lassen.

In [ ]:
flights_db %>% show_query()

Jetzt machen wir es komplexer. Im Folgenden Code sieht man, wie die dplyer-Funktionen `filter()` und `arrange()` in SQL´s `WHERE` und `ORDER BY` übersetzt werden:

In [ ]:
flights_db %>%
  filter(dest == "IAH") %>%
  arrange(dep_delay) %>%
  show_query()

`GROUP BY` funktioniert ähnlich wie `group_by()` von `dplyr` – es erzeugt eine Zusammenfassung und aggregiert anschließend die Daten:

In [ ]:
flights_db %>%
  group_by(dest) %>%
  summarize(dep_delay = mean(dep_delay, na.rm = TRUE)) %>%
  show_query()

An dieser Stelle sehen wir beispielsweise schon, wie `dbplyr` die Funktion `mean()` innerhalb des `summarize()` in die SQL-Funktion `AVG()` übersetzt. Keine Sorge, ihr müsst an dieser Stelle noch nicht alles lesen können. Dafür schauen wir uns gleich die einzelnen Befehle im Detail an.

Es gibt zwei wichtige Unterschiede zwischen `dplyr`-Befehlen und `SELECT` clauses:

* In SQL ist die Groß- und Kleinschreibung egal. Es ist möglich, `select`, `SELECT` oder sogar `sElEcT` zu schreiben. Eine gängige Notation ist jedoch, SQL keywords komplett als uppercase zu schreiben, also `SELECT`.
* In SQL ist jedoch die Reihenfolge wichtig. Diese fünf clauses müssen immer in der Reihenfolge `SELECT`, `FROM`, `WHERE`, `GROUP BY`, `ORDER BY` geschrieben werden, auch wenn das intuitiv nicht der Reihenfolge der Ausführung entspricht.

> An dieser Stelle sei auch erwähnt, dass SQL zwar eine absolute Standardsprache darstellt, es aber trotzdem sehr komplex werden kann und es viele unterschiedliche Datenbanktypen gibt, die dann wiederum feine Unterschiede untereinander haben. Für uns ist das erstmal nicht weiter schlimm, weil `dbplyr` jeweils versucht, je nach DBMS richtig zu übersetzen.

### `SELECT`

`SELECT` ist der Hauptteil jeder Datenabfrage und führt die Arbeit von `select()`, `mutate()`, `rename()` `relocate()`und`summarise()` aus.

`select()`, `rename()` und `relocate()` werden auch relativ direkt zu `SELECT` übersetzt:

In [ ]:
planes_db %>%
  select(tailnum, type, manufacturer, model, year) %>%
  show_query()

planes_db %>%
  select(tailnum, type, manufacturer, model, year) %>%
  rename(year_built = year) %>%
  show_query()

planes_db %>%
  select(tailnum, type, manufacturer, model, year) %>% 
  relocate(manufacturer, model, .before = type) %>%
  show_query()

An diesem Beispiel könnt ihr auch sehen, wie SQL Spalten umbenennt. Das umbenennen wird in SQL als *aliasing* und funktioniert mithilfe des Operators `AS`.

> Außerdem werden in dem obigen Beispiel die Spalten "year" und "type" in Anführungszeichen geschrieben. Das liegt einfach daran, dass beide Wörter sogenannte *reserved words* in `duckdb` sind. Innerhalb des Pakets stehen diese Worte für bestimmte Befehle. `dbplyr` ist schlau genug, diese mithilfe der Anführungszeichen als Spaltennamen zu kennzeichnen, sodass es keine Verwirrung gibt.

Die Übersetzung von `mutate()` funktioniert relativ ähnlich:

In [ ]:
flights_db %>%
  mutate(
    speed = distance / (air_time / 60)
  ) %>%
  show_query()


_____
<div style="background-color: #efe3fd"><h1>Präsenzteil</h1></div> 


## Das Zusammenspiel von SQL und `dbplyr`

### `FROM`

Mithilfe von `FROM` wird die Quelle der Abfrage definiert. In unserem sehr einfachen Anwendungsfall wird dort immer nur die Tabelle abgerufen, auf die wir zugreifen können. Kompliziertere Beispiele lassen wir hier aus. Aber natürlich wäre es möglich gleichzeitig Daten aus verschiedenen Tabellen abzufragen.

### `GROUP BY`

`group_by()` wird direkt zu `GROUP BY` übersetzt. `summarise()` verschwindet in gewisser Weise, die Funktionen innerhalb von `summarise()` werden aber das `SELECT` clause verschoben und übersetzt:

In [ ]:
flights_db %>%
  group_by(dest) %>%
  summarise(
    n = n(),
    avg_delay = mean(dep_delay, na.rm = TRUE)
  ) %>%
  show_query()

Wir werden später nochmal kurz darauf eingehen, was hier mit `n()` und `mean()` passiert.

### `WHERE`

`filter()` wird direkt in den `WHERE` clause übersetzt:

In [ ]:
flights_db %>%
  filter(dest == "IAH" | dest == "HOU") %>%
  show_query()

flights_db %>%
  filter(arr_delay > 0 & arr_delay < 20) %>%
  show_query()

Wichtige Details zu Operatoren in SQL:

* `|` in R entspricht `OR` in SQL
* `&` in R entspricht `AND` in SQL
* SQL benutzt `=` für Vergleiche und nicht `==` wie in R! Das kann ziemlich verwirrend sein.
* SQL benutzt zwingend `''` um Strings zu kennzeichnen, und nicht `""`. In SQL werden die doppelten Anführungszeichen `""` dafür benutzt, Variablen zu kennzeichnen!
* `%in%` kann über `IN` realisiert werden:

In [ ]:
flights_db %>%
  filter(dest %in% c("IAH", "HOU")) %>%
  show_query()

SQL benutzt `NULL` anstelle von `NA`. `NULL`-Werte verhalten sich analog wie `NA`-Werte. Außerdem sind einige Details anders, auf die wir an dieser Stelle aber nicht eingehen werden. Grundsätzlich könnt ihr die gleichen Funktionen in `dbplyr` verwenden, wie ihr sie auch für `NA` in R verwenden würdet:

In [ ]:
flights_db %>%
  filter(!is.na(dep_delay)) %>%
  show_query()

Diese Abfrage zeigt auch, warum `dbplyr` manchmal nicht ganz so gut funktioniert. Der SQL Code ist zwar korrekt, allerdings viel komplizierter, als er sein müsste und deshalb ist er schwer nachzuvollziehen. Stattdessen könnte man auch einen einfacheren Code schreiben:

```
WHERE "dep_delay" IS NOT NULL
```

Zum Glück gibt es Dokumentationen, in denen du alle wichtigen Details nachschlagen kannst. Deshalb musst du an dieser Stelle nicht alle Operatoren von SQL auswendig lernen. Trotzdem ist es hilfreich, wenn ihr diese Funktionen und Operatoren schon einmal gesehen habt. Generell unterscheiden sich verschiedene Programmiersprachen ohnehin oft in den konkreten Befehlen, die grundsätzliche Logik lernt ihr aber bereits hier kennen und könnt diese dann transferieren.

### `ORDER BY`

Um Reihen zu ordnen, können wir in SQL den Befehl `ORDER BY` benutzen, der direkt von `arrange()` übersetzt wird. An dem folgenden Code-Beispiel sieht man ganz gut, wie auch Argumente übersetzt werden, z. B. von `desc()` zu `DESC`. Tatsächlich sind viele Funktionsnamen von `dplyr` direkt von SQL inspiriert und daher an manchen Stellen gleich!

In [ ]:
flights_db %>%
  arrange(year, month, day, desc(dep_delay)) %>%
  show_query()

### Subqueries

Manchmal wird es nicht möglich sein, eine `dplyr` Pipe direkt in eine einzelne `SELECT`-Query zu übersetzen. Für diese Fälle gibt es sogenannte *subqueries*. Diese benutzen als Datenquelle (also hinter dem `FROM`-clause) eine weitere `SELECT`-Abfrage statt einer ganzen Tabelle.

Ein Hauptgrund für subqueries sind Hindernisse in SQL. Z.B. kann `SELECT` keine Spalten auswählen, die gerade erst erzeugt wurden. In einer `dplyr` Pipe wäre das möglich, sodass `dbplyr` dann subqueries verwendet, um die eingegebenen Pipes zu imitieren:

In [ ]:
flights_db %>%
  mutate(
    year1 = year + 1,
    year2 = year1 + 1
  ) %>%
    show_query

Manchmal wird es auch passieren, dass `dbplyr` eine subquery erzeugt, wo gar keine notwendig gewesen wäre. Auch `dbplyr` ist noch nicht vollständig auf Übersetzungen von `dplyr` in SQL trainiert. Daher ist es sinnvoll, bei einer weiteren Beschäftigung mit SQL auch andere Wissensquellen in Betracht zu ziehen!

**Fragen**
1. Wie wird `distinct()` in SQL übersetzt? Und wie `head()`?

2. Versucht die beiden SQL-queries im Kopf durchzudenken und zu überlegen, was diese tun. Anschließend reproduziert den Code mithilfe von `dbplyr`.

```
SELECT * 
FROM flights
WHERE dep_delay < arr_delay
```

```
SELECT *, distance / (airtime / 60) AS speed
FROM flights
```

3. Findet alle Flüge aus der Tabelle `flights_db`, die eine Verspätung von mehr als 30 Minuten hatten.

4. Verwendet `dbplyr`, um die durchschnittliche Verspätung (sowohl `dep_delay` als auch `arr_delay`) pro Fluggesellschaft `carrier` zu berechnen. Überlegt euch **vorher**, wie der dazugehörige SQL-Code aussehen wird und vergleicht dann eure Ergebnisse!

## Weitere Funktionen von R in SQL übersetzen

Bis hierhin haben wir grundlegende SQL-Befehle (clauses) für die Abfrage (query) von Daten kennengelernt. Natürlich kennt SQL als Befehle nicht nur die bisher kennengelernten, grundlegenden clauses. Wie sieht z.B. ein `mean(x)` innerhalb von `summarise()` in SQL aus? Was passiert wenn wir mit einzelnen Spalten arbeiten wollen, statt mit ganzen Tabellen?

Auch in diesem Abschnitt werden wir wieder `dplyr`-Code nach SQL übersetzen. Wie wir gleich sehen werden, haben manche Funktionen, wie z.B. `mean()`, relativ einfache Übersetzungen. Andere Funktionen wie `median()` sind sehr viel komplexer. Das liegt typischerweise daran, dass manche Befehle vor allem in der Statistik benutzt werden, für Datenbanken aber nicht unbedingt relevant sind, also auch kaum genutzt werden. 

> Wir haben leider nicht die Zeit, in aller Tiefe auf alle SQL-Befehle einzugehen. Keine Sorge, wenn ihr nicht direkt alles versteht – es geht hier wieder eher darum, dass ihr ein bisschen Gefühl für die Sprache bekommt und Begriffe schon einmal gesehen habt. Selbst, wenn ihr mal mit SQL arbeiten müsst, könnt ihr ja auch einfach `dbplyr` benutzen, und eure R-Skills anwenden, statt nochmal SQL zu lernen! Außerdem ergibt sich die Logik einer Programmiersprache auch manchmal mit der Zeit – je mehr man davon gesehen hat, desto einfacher wird es! Überfliegt den folgenden Abschnitt gern, und wenn ihr sehr motiviert seid, könnt ihr auch gern ein bisschen zu SQL recherchieren – über eine Suchmaschine oder auch direkt einen Chatbot!

Als Erstes schauen wir uns die beiden Funktionen `mean()` und `median()` innerhalb eines `summarise()`-Statements direkt einmal an:

In [ ]:
flights_db %>%
    group_by(year, month, day) %>%
        summarise(
        mean = mean(arr_delay, na.rm = TRUE),
        median = median(arr_delay, na.rm = TRUE),
        .groups = "drop"
    ) %>%
    show_query()

Wie wir sehen können, benutzt SQL den Befehl `AVG` für `mean()`. Die Darstellung eines Medians ist schon wesentlich komplizierter! 

`mutate()` können wir ebenfalls in SQL übersetzen. Diese ergibt dann eine sogenannte **window function**. Dafür wird der SQL-Befehl `OVER` benutzt, auf diesen wir an dieser Stelle nicht weiter eingehen wollen:

In [ ]:
flights_db %>%
    group_by(year, month, day) %>%
    mutate(
        mean = mean(arr_delay, na.rm = TRUE),
        .keep = "none"    
    ) %>%
    show_query()

`GROUP BY` wird in SQL ausschließlich für Zusammenfassungen benutzt. Wenn wir trotzdem aber gruppierte Ergebnisse ausgeben wollen, benutzen wir wieder `PARTITION BY` und `OVER`.

In [ ]:
flights_db %>%
    group_by(dest) %>%
    arrange(time_hour) %>%
    mutate(
        lead = lead(arr_delay),
        lag = lag(arr_delay),
        .keep = "none"
    ) %>%
    show_query()

SQL-Tabellen haben von Haus aus keine intrinsische Sortierung! Daher ist es wichtig, `arrange()` zu benutzen, um die Tabelle zunächst sortieren! Wenn `arrange()` nicht benutzt wird, könnten die Reihen jedes Mal in einer anderen Reihenfolge ausgegeben werden! Außerdem sehen wir, dass `ORDER BY` innerhalb der *window function* `OVER` nochmal benutzt wird! Die `ORDER BY` clause der Hauptabfrage wird nicht automatisch an *window functions* übergeben!

Bei der Auswahl von Fällen mithilfe von von `case_when()` – oder auch `if_else()` – wird `CASE WHEN` benutzt. Auch hier sehen wir wieder, wie `dplyr` von SQL namensgebend inspiriert wurde.

In [ ]:
flights_db %>%
    mutate(
    description = 
        case_when(
            arr_delay < -5 ~ "early", 
            arr_delay < 5 ~ "on-time",
            arr_delay >= 5 ~ "late"
        ),
        .keep = "none"
    ) %>%
    show_query()

`dbplyr` übersetzt auch gebräuchliche R-Funktionen, mit denen man Strings oder Datetime-Objekte manipulieren kann. Mehr darüber kannst du in der [Dokumentation](https://dbplyr.tidyverse.org/articles/translation-function.html) lernen! Die Übersetzungen von `dbplyr` sind sicherlich nicht perfekt und ersetzen bei häufiger Anwendung auch keine direkten Auseinandersetzung mit SQL, für den Anfang und einfache Zwecke sind sie für euch aber sicherlich hilfreich!

**Fragen**

5. Erzeugt eine neue Spalte in `flights_db`, die die Gesamtverspätung `total_delay` als Summe von `dep_delay` und `arr_delay` darstellt. Überlegt euch vorher, wie der dazugehörige SQL-Code aussehen wird, und vergleicht anschließend das Ergebnis damit.

6. Schaut euch die Dokumentation von [`between()`](https://dplyr.tidyverse.org/reference/between.html) and und überlegt euch, wie ihr mithilfe von `filter()` und [`between()`](https://dplyr.tidyverse.org/reference/between.html) alle Flüge aus `flights_db` auswählen könnt, bei denen die `airtime` zwischen 60 und 120 Minuten und die `distance` weniger als 500 Meilen beträgt. Schaut euch anschließend an, wie das ganze in SQL-Code übersetzt wird.

7. Was macht der folgende Code? Überlegt euch, wie das ganze in SQL aussehen wird und vergleicht anschließend.

```
flights_db %>%
  group_by(origin) %>%
  summarise(
    flights_count = n(),
    avg_dep_delay = mean(dep_delay, na.rm = TRUE),
    max_distance = max(distance, na.rm = TRUE)
  )
```

## Zusammenfassung

In dieser Sitzung haben wir einen ersten Einblick in die Programmiersprache SQL erhalten. Da wir im Rahmen dieser Übung keine Zeit haben, uns ausführlich mit SQL zu beschäftigen, haben wir das Paket `dbplyr` benutzt, um `dplyr`-Code in SQL zu übersetzen. SQL ist als meistverbreiteste Datenbanksprache wichtig, und es ist immer gut einen kleinen Überblick über eine so zentrale Sprache zu haben. Wenn ihr noch mehr über SQL lernen wollt, sind außerdem diese weiterführenden Links zu empfehlen:

* [SQL for Data Scientists](https://sqlfordatascientists.com/) von René M. P. Teate ist eine Einführung in SQL, die vor allem für Data Scientists geeignet ist. Sie enthält viele Beispiele für stark vernetzte Datensätze. Also so, wie sie auch in der Realität auftreten.
* [Practical SQL](https://www.practicalsql.com/) von Anthony DeBarros erzählt die Perspektive eines Datenjournalisten und geht dabei detaillierter darauf ein, wie man eigene Daten in eine Datenbank bekommt und sein eigenes DBMS betreibt.
* [SQL Island](https://sql-island.informatik.uni-kl.de/) ist ein Text-Adventure-Lernspiel für die Datenbanksprache SQL.

Aufgrund des zeitlichen Umfangs hatten wir in dieser Sitzung keine Zeit, auf sogenannte APIs einzugehen. Das sind Schnittstellen im Internet, über welche ebenfalls häufig Daten abgerufen werden – also quasi das Verbindungsstück zwischen uns und einer internen SQL-Datenbank. Die meisten Webseiten, die mit großen Datenmengen arbeiten, haben (oder hatten) ihre eigenen APIs (z.B. Twitter, Spotify, Telegram, etc). APIs werden im kommenden Sommersemester in der Datenwelten 2 Übung aufgegriffen werden. Ein einfaches Beispiel einer API findet sich noch im weiterführenden (optionalen) Teil.

## Weiterführend

### `JOIN` und weitere SQL-verbs

Wir haben bisher noch nicht die [joins](https://dplyr.tidyverse.org/reference/mutate-joins.html) von `dplyr` kennengelernt, es gibt aber in `dbplyr` und SQL ebenfalls "joins", die sehr praktisch sind. Allgemein gesagt ist ein "join" eine Funktion, mit der zwei Tabellen entlang einer oder mehrerer Variablen zusammengeführt werden können. Wir können beispielsweise die beiden Tabellen `flights_db` und `planes_db` zusammenführen. `flights_db` enthält Flugdaten, und `planes_db` zu den Flügen zugehörige Flugzeugmaschinendaten.

> In `dplyr` gibt es verschiedene [joins](https://dplyr.tidyverse.org/reference/mutate-joins.html), wie ihr auch in der Dokumentation nachlesen könnt. Darauf müssen wir an dieser Stelle aber nicht genauer eingehen.

Es folgt ein Beispiel, bei dem die Flugdaten mit den dazugehörigen Maschinendaten verknüpft werden:

In [ ]:
flights_db %>% 
    left_join(planes_db %>% rename(year_built = year), by = "tailnum") %>%
    select(tailnum, manufacturer, model)

In SQL sieht das ganze dann so aus:

In [ ]:
flights_db %>% 
    left_join(planes_db %>% rename(year_built = year), by = "tailnum") %>%
    show_query()

Neben `JOIN` kennt SQL auch noch `INNER JOIN`, `RIGHT JOIN` und `FULL JOIN`. Joins sind eine wichtige Funktion, wenn ihr mit Datenbanken arbeitet, da in Datenbanken die Tabellen oft einzelne Inhalte enthalten und erst miteinander verknüpft (joined) werden müssen. 

Ein praktisches Paket ist hierfür [`dm`](https://dm.cynkra.com/), mit welchem ihr direkt die Verbindungen innerhalb einer Datenbank anzeigen lassen könnt.

`dbplyr` kennt viele weitere SQL-verbs, welche ihr in der [Referenz](https://dbplyr.tidyverse.org/reference/) nachlesen könnt.

### APIs

Rein der Vollständigkeit halber seien hier noch APIs erwähnt, sogenannte Application Programming Interfaces, welche Schnittstellen übers Internet zu Datenbanken darstellen. APIs sind eine gängige Technik in der Informatik, auf die wir hier gar nicht im Detail eingehen wollen. Sie ermöglichen es uns, übers Internet Datenbanken anzuzapfen, um z.B. automatisiert Daten abzurufen.

Ein Mini-Beispiel für eine API ist [Open Notify taking NASA data](http://open-notify.org/), ein Open Source Projekt, welches eine API für NASA-Daten anbietet. In R können wir u.a. das Paket `jsonlite` benutzen, um über eine API Daten abzurufen:

In [ ]:
pacman::p_load(jsonlite)

Über die Adresse http://api.open-notify.org/iss-now.json können wir z.B. die aktuelle Adresse der ISS abrufen.

In [ ]:
jsonlite::fromJSON("http://api.open-notify.org/iss-now.json") %>%
    head()

Die von der API erhaltene Nachricht enthält die latitude und longitude der ISS, einen Zeitstempel sowie eine Benachrichtigung, ob die Abfrage von uns erfolgreich war.

Ein anderes Beispiel ist http://api.open-notify.org/astros.json, über welche wir die momentan sich auf der ISS befindlichen Personen sowie deren Anzahl abrufen können:

In [ ]:
jsonlite::fromJSON("http://api.open-notify.org/astros.json") %>%
    head()